"""
This script is used to caclulate the Event Mean Concentration (EMC).
The inputs are .csv files containing concentration and flow after linear interpolation.
"""

In [1]:
import pandas as pd
import numpy as np
from utils.concentration import rainfall_events, emc_cal, conc_interpolate, event_emc

# read the discrete storm events
# Read daily loads and flow
# Read hourly loads and flow
from common_settings import obspath, outpath, events_name, \
    obs_events, day_load_flow, hour_load_flow, conct_name, modpath

## Produce the event mean concentration of obs and mod

In [5]:
# Calculate EMC for low-frequency data
cols = [col for col in day_load_flow.columns if ('Load' in col) or ('Flow(ML)' in col)]
index_range1 = [1, 60]
index_range2 = [60, obs_events.shape[0]+1]

In [3]:
obs_events = event_emc(obs_events, day_load_flow, index_range1, cols[0], cols[1], 
    time_scale='d', multiplier=1e3)

In [4]:
# Calculate EMC for high-frequency data
cols = [col for col in hour_load_flow.columns if ('Load' in col) or ('ML' in col)]
index_range2 = [60, obs_events.shape[0]+1]
loads_col = cols[1]; flow_col = cols[0]
obs_events = event_emc(obs_events, hour_load_flow, index_range2, loads_col, flow_col, 
    time_scale='h', multiplier=1)
obs_events.to_csv(outpath + events_name, index='ID')

In [ ]:
# read the discrete storm events
modpath = '../data/mod/'
filename = 'storm_event.csv'
events = rainfall_events(f'{modpath}{filename}')

# Calculate EMC for modeling data
mod_fl_fn = 'DIN_flow.csv'
load_flow = pd.read_csv(modpath + mod_fl_fn, index_col='Date')
load_flow.index = pd.to_datetime(load_flow.index)
cols = [col for col in load_flow.columns if ('Load' in col) or ('ML' in col)]
index_range = [1, events.shape[0]]
loads_col = cols[0]; flow_col = cols[1]

events = event_emc(events, load_flow, index_range, loads_col, flow_col, 
    time_scale='d', multiplier=1)
events.dropna(axis=0, inplace=True)
# events.to_csv(f'{outpath}DIN_{filename}', index='ID')

## Produce the Normalized cumulative ratio of loads and flow 

In [2]:
from utils.concentration import cumulative_lq, excel_save
from utils.signatures import update_cumul_df, load_flow_loc

### calculate the daily data for double mass plot (Q-L)

In [10]:
time_ranges = [[f'{year}/7/1', f'{year+1}/6/30'] for year in range(2009, 2020)]
# time_ranges = obs_events.loc[:, ['start', 'end']].values
double_mass_ratio = {}

In [11]:
for ii in range(0, len(time_ranges)-2):
# for ii in range(index_range1[0]-1, index_range1[1]-1):
    df_temp = load_flow_loc(time_ranges[ii], day_load_flow, timestep='d')
    df_temp = update_cumul_df(df_temp, df_temp.values[:, 0], df_temp.values[:, -1])
    double_mass_ratio[f'obs_year_{ii}'] = df_temp

In [12]:
# save outputs into one excel
fn = outpath +'obs_year_cumulative_ratio_day.xlsx'
excel_save(double_mass_ratio, fn, True)

### calculate the hourly data for double mass plot (Q-L)

In [15]:
double_mass_ratio = {}
for ii in range(index_range2[0]-1, index_range2[1]-1):
# for ii in range(9, len(time_ranges)):
    df_temp = load_flow_loc(time_ranges[ii], hour_load_flow, timestep='h')
    df_temp = update_cumul_df(df_temp, df_temp.values[:, -1], df_temp.values[:, 0])
    double_mass_ratio[f'obs_storm_{ii}'] = df_temp

In [16]:
# save outputs into one excel
fn = outpath +'obs_storm_cumulative_ratio_hour.xlsx'
excel_save(double_mass_ratio, fn)

### calculate the modeling data for double mass plot (Q-L)

In [64]:
modpath = '../data/mod/'
filename = 'storm_event.csv'
mod_events = rainfall_events(f'{modpath}{filename}')

In [65]:
# Calculate EMC for modeling data
mod_fl_fn = 'DIN_flow.csv'
mod_load_flow = pd.read_csv(modpath + mod_fl_fn, index_col='Date')
mod_load_flow.index = pd.to_datetime(mod_load_flow.index)
cols = [col for col in mod_load_flow.columns if ('Load' in col) or ('ML' in col)]
index_range = [1, mod_events.shape[0]]
loads_col = cols[0]; flow_col = cols[1]

In [38]:
double_mass_ratio = {}
time_ranges = [[f'{year}-07-01', f'{year+1}-06-30'] for year in range(2009, 2014)]
# time_ranges = mod_events.loc[:, ['start', 'end']].values
# for ii in range(index_range[0], index_range[1]):
for ii in range(len(time_ranges)):
    df_temp = load_flow_loc(time_ranges[ii], mod_load_flow, timestep='d')
    df_temp = update_cumul_df(df_temp, df_temp.values[:, 0], df_temp.values[:, -1])
    double_mass_ratio[f'mod_storm_{ii}'] = df_temp

In [39]:
# save results
fn = outpath +'mod_year_cumulative_ratio_day.xlsx'
excel_save(double_mass_ratio, fn)

### Calculate event load coefficients

### Event loads for obs

In [13]:
obs_event_fn = 'obs_storm_event'
obs_events = pd.read_csv(f'{outpath}{obs_event_fn}.csv', index_col = 'ID') 
time_ranges = [[f'{year}/7/1', f'{year+1}/6/30'] for year in range(2009, 2020)]

In [14]:
# for each year, calculate the yearly loads 
year_loads = {}
# obs daily data
for tt in time_ranges[0:-2]:
    df = load_flow_loc(tt, day_load_flow, timestep='d')
    year_loads[tt[0][0:4]] = np.round(df.values[:, 0].sum(), 2)  
    
# obs hourly data
for tt in time_ranges[-2:]:
    df = load_flow_loc(tt, hour_load_flow, timestep='h')
    year_loads[tt[0][0:4]] = np.round(df.values[:, 0].sum(), 2)  

In [15]:
# The event load coefficients
for ii in range(1, index_range1[1]):
    df_event = load_flow_loc(obs_events.loc[ii, 'start':'end'].values, day_load_flow, timestep='d')
    ymd= pd.to_datetime(obs_events.loc[ii, 'start'])
    month = ymd.month; year = ymd.year
    if month < 7:
        obs_events.loc[ii, 'event_load_coefficients'] = df_event.values[:, 0].sum() / year_loads[str(year - 1)]
    else:
        obs_events.loc[ii, 'event_load_coefficients'] = df_event.values[:, 0].sum() / year_loads[str(year)]
        
for ii in range(index_range2[0], index_range2[1]):
    df_event = load_flow_loc(obs_events.loc[ii, 'start':'end'].values, hour_load_flow, timestep='h')
    ymd= pd.to_datetime(obs_events.loc[ii, 'start'])
    month = ymd.month; year = ymd.year
    if month < 7:
        obs_events.loc[ii, 'event_load_coefficients'] = df_event.values[:, 0].sum() / year_loads[str(year-1)]
    else:
        obs_events.loc[ii, 'event_load_coefficients'] = df_event.values[:, 0].sum() / year_loads[str(year)]

In [16]:
obs_events.to_csv(f'{outpath}{obs_event_fn}.csv')

### Event loads for mod

In [72]:
mod_event_fn = 'mod_NO3_flow'
mod_events = pd.read_csv(f'{outpath}{mod_event_fn}.csv', index_col = 'ID') 
time_ranges = [[f'{year}/7/1', f'{year+1}/6/30'] for year in range(2009, 2014)]

In [70]:
# for each year, calculate the yearly loads 
mod_loads = {}
# obs daily data
for tt in time_ranges:
    df = load_flow_loc(tt, mod_load_flow, timestep='d')
    mod_loads[tt[0][0:4]] = np.round(df.values[:, 0].sum(), 2)  

In [74]:
# The event load coefficients
for ii in mod_events.index:
    df_event = load_flow_loc(mod_events.loc[ii, 'start':'end'].values, mod_load_flow, timestep='d')
    ymd= pd.to_datetime(mod_events.loc[ii, 'start'])
    month = ymd.month; year = ymd.year
    if month < 7:
        mod_events.loc[ii, 'event_load_coefficients'] = df_event.values[:, 0].sum() / mod_loads[str(year - 1)]
    else:
        mod_events.loc[ii, 'event_load_coefficients'] = df_event.values[:, 0].sum() / mod_loads[str(year)]

In [76]:
mod_events.to_csv(f'{outpath}{mod_event_fn}.csv')